<a href="https://colab.research.google.com/github/devu-13here/ML_assignmentt/blob/main/Assignment_3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import files
#uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.read_csv('AWCustomers.csv')


In [ ]:
# Display the column names in your DataFrame
# (The to_string() method will display the entire DataFrame as a string without truncation.)
print(df.head().to_string())

In [ ]:

selected_columns = ['City', 'StateProvinceName', 'CountryRegionName', 'PostalCode', 'BirthDate','Education', 'Occupation', 'Gender', 'MaritalStatus','HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren','YearlyIncome']

# checking for no of null values present
null = df.isnull().sum()
print(null)

In [ ]:
df_selected = df[selected_columns]
print(df_selected.head().to_string())

In [ ]:
# Q1.c datatype(Discrete/Continuous) & measurement(Nominal, Ordinal, Interval, Ratio) for each attribute in df_selected

df_Types_Scales = {'Attribute': [], 'Type': [], 'Scale': []}

df_Types_Scales['Attribute'].extend(['City', 'StateProvinceName', 'CountryRegionName', 'PostalCode', 'BirthDate','Education', 'Occupation', 'Gender', 'MaritalStatus','HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren','YearlyIncome'])
df_Types_Scales['Type'].extend(['Discrete', 'Discrete', 'Discrete', 'Discrete', 'Continuous','Discrete', 'Discrete', 'Discrete', 'Discrete','Discrete', 'Discrete', 'Discrete', 'Discrete','Continuous'])
df_Types_Scales['Scale'].extend(['Nominal', 'Nominal', 'Nominal', 'Nominal', 'Interval','Ordinal', 'Nominal', 'Nominal', 'Nominal','Nominal', 'Ratio', 'Ratio', 'Ratio','Ratio'])

# Create a DataFrame to display the information
df_Types_Scales = pd.DataFrame(df_Types_Scales)

# Display the data type and scale information
print(df_Types_Scales.to_string(index=False))

In [ ]:
# In df_selected there are no colms with null values, so handing the null values in df

# for a col having too many null values, drop it

df = df.drop(columns=['Title', 'Suffix', 'AddressLine2'])
print(df.head().to_string())


In [ ]:
# for categorical data(nominal, ordinal) Fill null values with the mode or a placeholder(like unknown)

df['MiddleName'].fillna("Unknown", inplace=True)
df['Occupation'].fillna(df['Occupation'].mode()[0], inplace = True)
print(df.head().to_string())


In [ ]:

# for continuous data(interval, ratio): Fill null values with mean, median, or placeholder like 0

df['YearlyIncome'].fillna(df['YearlyIncome'].mean(), inplace = True)
print(df.head().to_string())

In [ ]:
# verify if there are no null values

null = df.isnull().sum()
print(null)

In [ ]:
# II. b. Normalization
# (Normalization is the process of scaling numerical data to a common range without distorting differences in the ranges of values.)

# finding numerical colms
numerical_colm = df_selected.select_dtypes(include=['int64', 'float64']).columns

print("Numerical cols: ", numerical_colm)

In [ ]:
#  normalize using Min-Max Scaling

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()  #initialse the minMaxScaler

numerical_colms = ['HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
       'TotalChildren', 'YearlyIncome']
df_selected[numerical_colms] = scaler.fit_transform(df_selected[numerical_colms])

print(df_selected.head().to_string())

In [ ]:
print(df_selected[numerical_colms].describe().to_string())

In [ ]:

 #normalize yearly income using Z-score normalizn. (Standadization)

 from sklearn.preprocessing import StandardScaler
 scaler = StandardScaler()

 colm = 'YearlyIncome'
 df_selected[colm] = scaler.fit_transform(df_selected[[colm]])

 df_selected[colm].head()


In [ ]:
# II.c. Discretization (binning) is the process of converting continuous data into discrete bins or intervals.

bins = [0, 40000, 80000, 120000]
labels = ['Low', 'Medium', 'High']

df_selected['IncomeCategory'] = pd.cut(df_selected['YearlyIncome'], bins=bins, labels=labels, include_lowest=True)

print(df_selected[['YearlyIncome', 'IncomeCategory']].head().to_string())


In [ ]:
# II.d Standardization/Normalization

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_selected['TotalChildren_standardized'] = scaler.fit_transform(df_selected[['TotalChildren']])

print(df_selected[['TotalChildren', 'TotalChildren_standardized']].head().to_string())


In [ ]:
# II.e. One Hot Encoding

# Perform one hot encoding on 'Education' and 'Occupation'
df_selected = pd.get_dummies(df_selected, columns=['Education', 'Occupation'], drop_first=True)

# Verify the result by printing the first few rows
print(df_selected.head().to_string())

In [ ]:
# III.
# Step 1: Transform the Data
# Numeric Attributes: Standardize the numeric attributes.
# Nominal Attributes: Use one-hot encoding for nominal attributes.
# Step 2: Calculate Similarity Metrics
# Let's assume you want to compare the first two rows (row indices 0 and 1).

import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import pairwise_distances, jaccard_score
from sklearn.metrics.pairwise import cosine_similarity  # Corrected import


# Nominal attributes based on your df_selected DataFrame
nominal_attributes = ['City', 'StateProvinceName', 'CountryRegionName', 'Gender', 'MaritalStatus',
                      'Education_Graduate Degree', 'Education_High School', 'Education_Partial College',
                      'Education_Partial High School', 'Occupation_Management', 'Occupation_Manual',
                      'Occupation_Professional', 'Occupation_Skilled Manual']

# Numeric attributes that need standardization
numeric_attributes = ['NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'TotalChildren_standardized']

# One-hot encode the nominal attributes
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_nominal = encoder.fit_transform(df_selected[nominal_attributes])

# Standardize the numeric attributes
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(df_selected[numeric_attributes])

# Combine the encoded nominal attributes and standardized numeric attributes
transformed_data = np.hstack((encoded_nominal, scaled_numeric))

# Select two rows to compare
row1 = transformed_data[0]
row2 = transformed_data[1]

# Calculate Simple Matching Similarity (binary nominal data is considered)
simple_matching_similarity = 1 - pairwise_distances([row1], [row2], metric='hamming')[0][0]
print(f'Simple Matching Similarity: {simple_matching_similarity}')

# Calculate Jaccard Similarity for binary nominal attributes
# For Jaccard, use only the binary (0/1) nominal data
jaccard_similarity = jaccard_score(row1[:encoded_nominal.shape[1]], row2[:encoded_nominal.shape[1]])
print(f'Jaccard Similarity: {jaccard_similarity}')

# Calculate Cosine Similarity on the entire transformed data (binary + standardized numeric)
cosine_sim = cosine_similarity([row1], [row2])[0][0]
print(f'Cosine Similarity: {cosine_sim}')
